Code to train a model with the specifications that worked best after the grid search on the input data and produce and save an output to the desginated directory

In [26]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils, regularizers

In [2]:
os.getcwd()

'/Users/samklauer/GitHub/PawseySummer2024/Full Code'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'/Users/samklauer/GitHub/PawseySummer2024'

In [5]:
def fasta_reader(directory):
    seqs = []
    for file in os.listdir(directory):
        seq = ''
        filepath = directory + file
        #print(file)
        f = open(filepath, 'r')
        if not file.startswith('.'):
            for i in f:
                if i[0] == '>':
                    if seq != '':
                        seqs.append(seq)
                        seq = ''
                else:
                    seq += i.strip()
            if seq != '':
                seqs.append(seq)
    return(seqs)

In [6]:
segments = fasta_reader('./segment database/segments/')
nonseg = fasta_reader('./segment database/NCnonsegmentedsingle/')

In [7]:
def one_hot_encode(sequence):
    encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'U':[0, 0, 0, 1], 'N':[0,0,0,0]
    , 'M':[0,0,0,0], 'R':[0,0,0,0], 'W':[0,0,0,0], 'S':[0,0,0,0], 'Y':[0,0,0,0], 'K':[0,0,0,0], 'V':[0,0,0,0], 'H':[0,0,0,0], 'D':[0,0,0,0],
    'B':[0,0,0,0], 'E':[0,0,0,0], 'F':[0,0,0,0], 'I':[0,0,0,0], 'J':[0,0,0,0], 'L':[0,0,0,0], 'O':[0,0,0,0], 
    'P':[0,0,0,0], 'Q':[0,0,0,0], 'X':[0,0,0,0], 'Z':[0,0,0,0]}
    encoded_sequence = []

    for nucleotide in sequence:
        if encoding.get(nucleotide):
            encoded_sequence.append(encoding.get(nucleotide))
        else:
            encoded_sequence.append([0,0,0,0])

    feature_length = 20000

    while len(encoded_sequence) <20000:
        encoded_sequence.append([0,0,0,0])

    return encoded_sequence

In [13]:
one_hot_non = []
for i in nonseg:
    one_hot_non.append(one_hot_encode(i))

In [14]:
one_hot_seg = []
for i in segments:
    one_hot_seg.append(one_hot_encode(i))

In [15]:
y = [1] * len(one_hot_seg) + [0] * len(one_hot_non)

In [16]:
X = one_hot_seg + one_hot_non

KeyboardInterrupt: 

In [17]:
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(int(.8*len(one_hot_seg))):
    X_train.append(one_hot_seg[i])
    y_train.append([1])
for i in range(int(.8*len(one_hot_seg)), len(one_hot_seg)):
    X_test.append(one_hot_seg[i])
    y_test.append([1])
for i in range(int(.8 * len(one_hot_non))):
    X_train.append(one_hot_non[i])
    y_train.append([0])
for i in range(int(.8 * len(one_hot_non)), len(one_hot_non)):
    X_test.append(one_hot_non[i])
    y_test.append([0])

In [27]:
def model_builder(x, y, z):
    model = models.Sequential()
    model.add(layers.Conv1D(32, 2, activation='relu', input_shape=(20000,4), kernel_regularizer=regularizers.L1L2(l1=x, l2=x),
        activity_regularizer=regularizers.L2(y)))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(z))
    model.add(layers.Conv1D(32, 2, activation='relu', kernel_regularizer=regularizers.L1L2(l1=x, l2=x),
        activity_regularizer=regularizers.L2(y)))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(z))
    model.add(layers.Conv1D(32, 2, activation='relu', kernel_regularizer=regularizers.L1L2(l1=x, l2=x),
        activity_regularizer=regularizers.L2(y)))
    model.add(layers.MaxPooling1D())
    model.add(layers.Flatten())
    model.add(layers.Dropout(z))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

In [28]:
dropout = .5
kern = .0001
act = .0001 

In [29]:
helper =  'finalmodel.keras'

In [30]:
model = model_builder(kern, act, dropout)
model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])
            
model.fit(X_train, y_train, batch_size = 128, epochs=10,shuffle = True)

model.save(helper)

test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/10


/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 1s/step - loss: 2.0107 - accuracy: 0.4375
Epoch 2/10
1/1 [==============================] - 0s 400ms/step - loss: 2.4296 - accuracy: 0.6250
Epoch 3/10
1/1 [==============================] - 0s 372ms/step - loss: 3.2412 - accuracy: 0.6875
Epoch 4/10
1/1 [==============================] - 0s 389ms/step - loss: 2.8175 - accuracy: 0.6875
Epoch 5/10
1/1 [==============================] - 0s 402ms/step - loss: 1.8065 - accuracy: 0.7500
Epoch 6/10
1/1 [==============================] - 0s 383ms/step - loss: 1.6668 - accuracy: 0.7500
Epoch 7/10
1/1 [==============================] - 0s 406ms/step - loss: 1.7991 - accuracy: 0.7500
Epoch 8/10
1/1 [==============================] - 0s 377ms/step - loss: 1.6826 - accuracy: 0.7500
Epoch 9/10
1/1 [==============================] - 0s 447ms/step - loss: 1.4086 - accuracy: 0.8125
Epoch 10/10
1/1 [==============================] - 0s 189ms/step - loss: 1.0696 - accuracy: 0.5000
